# Analyse des données extraite via Graal

## Ouverture de data.csv et insertion dans df

In [121]:
import pandas as pd
import numpy as np
import seaborn as sns
import re

#Fonction pour clean le data (le plus générale possible)
def cleanData(df,columns,vide):
    #on ajoute le header
    df.columns =columns
    #On récupère que les valeurs entre guillemets
    df= df.applymap(lambda x: re.search(r'(^.*"(.*)\")',str(x)).group(2))

    #Passage NaN au valeur aillant le mot vide
    for column in df.columns[0:]:
        df.loc[df[column]==vide,column]=np.nan
    return df


        
#on transforme en dataframe
df = pd.read_csv("data.csv", header=None)
#Définition du header
columns = ["boat","embarked","sex","age","firstname","lastname","parch","sibsp","ticket","pclass"]    
#indication du mot vide (modulable selon si c'est ?, Unknown,null, NULL etc)
vide = 'null'

df = cleanData(df,columns,vide)

df.head()


,boat,embarked,sex,age,firstname,lastname,parch,sibsp,ticket,pclass
0,NaN,S,male,NaN,Mr. Ingvald Olai Olsen,Hagland,0,1,65303,3
1,NaN,S,female,8.0,Miss. Torborg Danira,Palsson,1,3,349909,3
2,11,NaN,female,22.0,Miss. Alice,Cleaver,0,0,113781,1
3,9,NaN,F,24.0,Miss. Alice,Herman,2.0,1.0,220845,2
4,NaN,S,male,20.0,Mr. Domingos Fernandeo,Coelho,0,0,SOTON/O.Q. 3101307,3


In [118]:
##

In [12]:
df.header

AttributeError: 'DataFrame' object has no attribute 'header'